In [1]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
%%capture output
!python3 -m pip install git+https://github.com/dropbox/PyHive.git >/dev/null

In [3]:
from utility import Utility as util

In [4]:
import pandas as pd
pd.options.display.max_columns = None
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)
import warnings
import time
import os
#import pandasql as psql
from IPython.core.display import Markdown as md
warnings.filterwarnings('ignore')
#from libraries.project_reports import project_reports
#from libraries.utility import Utility as util
mutil = util()
img_path = os.path.join(mutil.get_this_dir(),"artifacts","images","irwin_analytics_small.txt")
img_text_data = mutil.get_data_from_file(img_path)

img_tag = "<img align=\"left\" src=\"data:image/png;base64,{}\" />".format(img_text_data)
a_tag = "<a href=\"https://www.irwinanalytics.com\" target=\"https://www.irwinanalytics.com\">{}</a>".format(img_tag)
#print(img_text_data)

from IPython.core.display import display, HTML, Markdown, Latex
display(HTML(
    '<style>'
        '#notebook { padding-top:0px !important; } ' 
        '.container { width:100% !important; } '
        '.end_space { min-height:0px !important; } '
        '.end_space { min-height:0px !important; } '
        '.prompt {width: 0px; min-width: 0px; visibility: collapse } '
        '.parent{'
        '    display: grid;'
        '    grid-template-columns: 1fr 1fr;'
        '    column-gap: 5px;'
        '}    '
    '</style>'
))


my_html = "<hr/>".format(img_tag,a_tag)
display(HTML(my_html))
my_html = "<div class='parent'><div>{}</div></div>".format(a_tag)
display(HTML(my_html))

## Using Presto to Generate Reports

To generate reports for those interested in application usage, we can use Presto via the PyHive connector to load our Hive tables into Pandas dataframes, transform the dataframes to answer our business questions, and then publish the reports as JSON files.

Before we start, let's define two business questions that we would like to answer:

1. What are all the counts per event type?
2. What are all the parameters that were given for the `user` parameter?

First, let's install the PyHive library.

Next, let's use PyHive to connect to Presto in code using the port that we exposed in our Docker Compose file. Once we connect, we can run a simple query to see all the tables that are created in Hive.

In [5]:
from pyhive import presto
import pandas as pd

presto_conn = presto.connect(
    host='presto',
    port=8080 # Exposed Presto port (see docker compose file)
)

pd.read_sql_query("SHOW TABLES", presto_conn)

,Table
0,all_events
1,event_parameters


Now let's run a query to get all of the data from the `event_parameters` table and load it into a Pandas dataframe.

In [6]:
# https://stackoverflow.com/questions/55988436/how-to-convert-a-presto-query-output-to-a-python-data-frame
event_parameters = pd.read_sql_query("SELECT * from event_parameters", presto_conn)
event_parameters.tail()

,raw_event,timestamp,accept,host,user_agent,event_id,parameter_name,parameter_value
5519,"{""event_id"": ""33e8af4a-d4c6-457b-bd1f-8711496e...",2023-08-29 01:25:49.762,None,None,None,33e8af4a-d4c6-457b-bd1f-8711496e8066,user,don
5520,"{""event_id"": ""aa0e0d73-1782-4e84-ba08-ba4d2565...",2023-08-29 01:25:49.764,None,None,None,aa0e0d73-1782-4e84-ba08-ba4d2565e02f,sword_type,Nightweaver
5521,"{""event_id"": ""aa0e0d73-1782-4e84-ba08-ba4d2565...",2023-08-29 01:25:49.764,None,None,None,aa0e0d73-1782-4e84-ba08-ba4d2565e02f,user,don
5522,"{""event_id"": ""2709d001-e9b7-4c66-9024-da7088b3...",2023-08-29 01:25:49.766,None,None,None,2709d001-e9b7-4c66-9024-da7088b33308,sword_type,Nightweaver
5523,"{""event_id"": ""2709d001-e9b7-4c66-9024-da7088b3...",2023-08-29 01:25:49.766,None,None,None,2709d001-e9b7-4c66-9024-da7088b33308,user,don


Let's do the same thing for the data in the `all_events` table.

In [7]:
all_events = pd.read_sql_query("SELECT * from all_events", presto_conn)
all_events.tail()

,raw_event,timestamp,accept,host,user_agent,event_id,event_type
0,"{""event_id"": ""6bae4a42-05e7-4347-a86a-9997a212...",2023-08-29 01:25:27.807,*/*,user2.att.com,ApacheBench/2.3,6bae4a42-05e7-4347-a86a-9997a212c4df,get_credit
1,"{""event_id"": ""b2181a48-1ed0-4b8f-881a-850adc8e...",2023-08-29 01:25:27.809,*/*,user2.att.com,ApacheBench/2.3,b2181a48-1ed0-4b8f-881a-850adc8ed4f0,get_credit
2,"{""event_id"": ""304e567f-34b8-4104-9746-c5a48d5a...",2023-08-29 01:25:27.81,*/*,user2.att.com,ApacheBench/2.3,304e567f-34b8-4104-9746-c5a48d5a9a4b,get_credit
3,"{""event_id"": ""95d6a70b-028b-447e-8965-90899939...",2023-08-29 01:25:27.812,*/*,user2.att.com,ApacheBench/2.3,95d6a70b-028b-447e-8965-90899939ae81,get_credit
4,"{""event_id"": ""94fe5e2b-2e44-42e1-b91e-6b8baad4...",2023-08-29 01:25:27.813,*/*,user2.att.com,ApacheBench/2.3,94fe5e2b-2e44-42e1-b91e-6b8baad4bd60,get_credit


Now let's try to answer business question #1 - What are all the counts per event type? We can do this using a simple groupby statement on our `all_events` dataframe and then writing the output to a JSON file titled `event_type_count.json`.

In [8]:
event_type_count = all_events.groupby('event_type').size()
event_type_count.to_json("event_type_count.json", orient='columns')

Note that `event_type_count.json` should exist in your directory after running the code above.


In [9]:
all_events = pd.read_sql_query("select event_type, count(event_type) as event_count from all_events group by event_type", presto_conn)
all_events.head()

,event_type,event_count
0,get_credit,706
1,leave_guild,654
2,purchase_sword,923
3,join_guild,479




Now let's answer business question #2 - What are all the parameters that were given for the `user` parameter? We can do this by running a slightly more complex query on our dataframe and again writing the output to a JSON file.

In [10]:
# Question: What are all the parameters that were given for the `user` parameter?
user_parameter_count = event_parameters.where(event_parameters['parameter_name'] == 'user').groupby('parameter_value').size()
user_parameter_count.to_json("user_parameter_count.json", orient='columns')

events_by_user = pd.read_sql_query(" select     un.parameter_value as user,     et.event_type as event,    count(un.parameter_value) as user_event_count from     all_events et join      event_parameters un on      et.event_id = un.event_id and     un.parameter_name = 'user' where      un.parameter_name = 'user'\
and     et.event_id = un.event_id group by     un.parameter_value     ,et.event_type order by     et.event_type,     un.parameter_value ", presto_conn)

events_by_user.head(50)


,user,event,user_event_count
0,aastha,get_credit,112
1,ben,get_credit,227
2,don,get_credit,159
3,lise,get_credit,53
4,theresa,get_credit,155
5,aastha,join_guild,133
6,ben,join_guild,109
7,don,join_guild,116
8,lise,join_guild,11
9,theresa,join_guild,110


Now that we have answered the business questions and generated JSON reports with the answers, we can close our Presto connection.

In [11]:
presto_conn.close()